## Deliverable 2. Create a Customer Travel Destinations Map.

In [23]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import matplotlib.pyplot as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [ ]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/alexhatheway/Desktop/DataViz/Week 6 WeatherPy/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 1
What is the maximum temperature you would like for your trip? 30


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
29,29,Magadan,RU,59.5667,150.8000,11.10,59,2,7.23,clear sky
33,33,Upernavik,GL,72.7868,-56.1549,26.96,47,100,6.69,overcast clouds
64,64,Qaanaaq,GL,77.4840,-69.3632,26.15,68,100,11.25,overcast clouds
70,70,Barrow,US,71.2906,-156.7887,10.42,79,20,4.61,few clouds
86,86,Pevek,RU,69.7008,170.3133,9.18,81,100,4.07,overcast clouds
116,116,Zakamensk,RU,50.3741,103.2863,26.46,69,8,2.13,clear sky
123,123,Provideniya,RU,64.3833,-173.3000,28.47,78,100,27.81,overcast clouds
125,125,Tuktoyaktuk,CA,69.4541,-133.0374,19.40,93,75,8.05,broken clouds
142,142,Tiksi,RU,71.6872,128.8694,16.77,90,100,11.88,overcast clouds
145,145,Smolenka,RU,52.1352,113.4974,17.67,93,13,4.47,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna(how='any')

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
29,Magadan,RU,11.10,clear sky,59.5667,150.8000,
33,Upernavik,GL,26.96,overcast clouds,72.7868,-56.1549,
64,Qaanaaq,GL,26.15,overcast clouds,77.4840,-69.3632,
70,Barrow,US,10.42,few clouds,71.2906,-156.7887,
86,Pevek,RU,9.18,overcast clouds,69.7008,170.3133,
116,Zakamensk,RU,26.46,clear sky,50.3741,103.2863,
123,Provideniya,RU,28.47,overcast clouds,64.3833,-173.3000,
125,Tuktoyaktuk,CA,19.40,broken clouds,69.4541,-133.0374,
142,Tiksi,RU,16.77,overcast clouds,71.6872,128.8694,
145,Smolenka,RU,17.67,few clouds,52.1352,113.4974,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]['name']
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found. . . skipping.")

Hotel not found. . . skipping.
Hotel not found. . . skipping.
Hotel not found. . . skipping.
Hotel not found. . . skipping.
Hotel not found. . . skipping.
Hotel not found. . . skipping.
Hotel not found. . . skipping.
Hotel not found. . . skipping.
Hotel not found. . . skipping.
Hotel not found. . . skipping.
Hotel not found. . . skipping.


In [14]:
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
29,Magadan,RU,11.10,clear sky,59.5667,150.8000,Magadan hotel
33,Upernavik,GL,26.96,overcast clouds,72.7868,-56.1549,Gina's Guesthouse - Upernavik
64,Qaanaaq,GL,26.15,overcast clouds,77.4840,-69.3632,Qaanaaq Hotel
70,Barrow,US,10.42,few clouds,71.2906,-156.7887,King Eider Inn
86,Pevek,RU,9.18,overcast clouds,69.7008,170.3133,Severnoye Zoloto
116,Zakamensk,RU,26.46,clear sky,50.3741,103.2863,Gostinitsa Zakamna
125,Tuktoyaktuk,CA,19.40,broken clouds,69.4541,-133.0374,End of The Road Ltd.
142,Tiksi,RU,16.77,overcast clouds,71.6872,128.8694,Arktika
145,Smolenka,RU,17.67,few clouds,52.1352,113.4974,Russkaya Usadba v Zarechye
147,Nome,US,18.16,overcast clouds,64.5011,-165.4064,Dredge 7 Inn


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna(how='any')
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
29,Magadan,RU,11.10,clear sky,59.5667,150.8000,Magadan hotel
33,Upernavik,GL,26.96,overcast clouds,72.7868,-56.1549,Gina's Guesthouse - Upernavik
64,Qaanaaq,GL,26.15,overcast clouds,77.4840,-69.3632,Qaanaaq Hotel
70,Barrow,US,10.42,few clouds,71.2906,-156.7887,King Eider Inn
86,Pevek,RU,9.18,overcast clouds,69.7008,170.3133,Severnoye Zoloto
116,Zakamensk,RU,26.46,clear sky,50.3741,103.2863,Gostinitsa Zakamna
125,Tuktoyaktuk,CA,19.40,broken clouds,69.4541,-133.0374,End of The Road Ltd.
142,Tiksi,RU,16.77,overcast clouds,71.6872,128.8694,Arktika
145,Smolenka,RU,17.67,few clouds,52.1352,113.4974,Russkaya Usadba v Zarechye
147,Nome,US,18.16,overcast clouds,64.5011,-165.4064,Dredge 7 Inn


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "/Users/alexhatheway/Desktop/DataViz/Week 6 WeatherPy/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
plt.savefig('WeatherPy_vacation_map.png')

AttributeError: 'Figure' object has no attribute 'show'

<Figure size 432x288 with 0 Axes>